In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVR
from numpy import mean
import statistics
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def prediction_eval_function(mouse_pupil_csv, l23_csv,l4_csv,spiketrains_csv):

    mouse_pupil= pd.read_csv(mouse_pupil_csv)
    l23 = pd.read_csv(l23_csv)
    l4=pd.read_csv(l4_csv)
    spiketrains = pd.read_csv(spiketrains_csv)
    
    spiketrains.columns=spiketrains.columns.str.replace('V', '')
    neurons2_3=list(l23.loc[:,"Neurons"])
    neurons2_3=[str(x) for x in neurons2_3]
    neurons4=list(l4.loc[:,"Neurons"])
    neurons4=[str(x) for x in neurons4]
    n23=spiketrains.drop(columns=[col for col in spiketrains.columns if col not in neurons2_3])
    n4=spiketrains.drop(columns=[col for col in spiketrains.columns if col not in neurons4])
    n23_4=spiketrains.drop(columns=[col for col in spiketrains.columns if col not in neurons4 and col not in neurons2_3 ])
    result2_3 = pd.concat([mouse_pupil, n23], axis=1, join='inner')
    result4 = pd.concat([mouse_pupil, n4], axis=1, join='inner')
    result2_3_4=pd.concat([mouse_pupil, n23_4], axis=1, join='inner')
    result2_3.drop(['Frames','Timestamps','PupilX','PupilY'], axis=1, inplace=True) # dropping the cols I dont need
    result4.drop(['Frames','Timestamps','PupilX','PupilY'], axis=1, inplace=True) # dropping the cols I dont need
    result2_3_4.drop(['Frames','Timestamps','PupilX','PupilY'], axis=1, inplace=True) # dropping the cols I dont need
    
    pupil_mean_2_3_4=result2_3_4['PupilRadius'].mean()
    pupil_std_2_3_4=result2_3_4['PupilRadius'].std()
    
    pupil_mean_2_3=result2_3['PupilRadius'].mean()
    pupil_std_2_3=result2_3['PupilRadius'].std()
    
    pupil_mean4=result4['PupilRadius'].mean()
    pupil_std4=result4['PupilRadius'].std()
    
    
    result2_3['PupilRadius']=(result2_3['PupilRadius']-pupil_mean_2_3)/pupil_std_2_3
    result4['PupilRadius']=(result4['PupilRadius']-pupil_mean4)/pupil_std4
    result2_3_4['PupilRadius']=(result2_3_4['PupilRadius']-pupil_mean_2_3_4)/pupil_std_2_3_4


    X = result2_3.iloc[:, 1:].values
    y = result2_3.iloc[:, 0].values
    kf = KFold(n_splits = 12,shuffle=True, random_state=10)
    rmse_final= list()
    columns=result2_3.iloc[:, 1:].columns.tolist()
    coef_df_l23=pd.DataFrame(columns=columns)
    for train_ix, test_ix in kf.split(X):
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        model =LinearSVR(C=0.5, dual=True, epsilon=0.0, fit_intercept=True,
            intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=10000,
            random_state=None, tol=0.0001, verbose=0)
        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)
        rmse = mean_squared_error(test_y, y_pred)*(1/2.0)
        rmse_final.append(rmse)
        #print('> ', rmse)
        coef_df_l23.loc[len(coef_df_l23)] = model.coef_
    mean_rmse_l23= mean(rmse_final)
    mean_coef_df_l23=coef_df_l23.mean().to_frame()

    X = result4.iloc[:, 1:].values
    y = result4.iloc[:, 0].values
    kf = KFold(n_splits = 12,shuffle=True, random_state=10)
    rmse_final= list()
    columns=result4.iloc[:, 1:].columns.tolist()
    coef_df_l4=pd.DataFrame(columns=columns)
    for train_ix, test_ix in kf.split(X):
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        model =LinearSVR(C=0.5, dual=True, epsilon=0.0, fit_intercept=True,
            intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=10000,
            random_state=None, tol=0.0001, verbose=0)
        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)
        rmse = mean_squared_error(test_y, y_pred)*(1/2.0)
        rmse_final.append(rmse)
        #print('> ', rmse)
        coef_df_l4.loc[len(coef_df_l4)] = model.coef_
    mean_rmse_l4= mean(rmse_final)
    mean_coef_df_l4=coef_df_l4.mean().to_frame()

    X = result2_3_4.iloc[:, 1:].values
    y = result2_3_4.iloc[:, 0].values
    kf = KFold(n_splits = 12,shuffle=True, random_state=10)
    rmse_final= list()
    columns=result2_3_4.iloc[:, 1:].columns.tolist()
    coef_df_l2_3_4=pd.DataFrame(columns=columns)
    for train_ix, test_ix in kf.split(X):
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        model =LinearSVR(C=0.5, dual=True, epsilon=0.0, fit_intercept=True,
            intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=10000,
            random_state=None, tol=0.0001, verbose=0)
        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)
        rmse = mean_squared_error(test_y, y_pred)*(1/2.0)
        rmse_final.append(rmse)
        #print('> ', rmse)
        coef_df_l2_3_4.loc[len(coef_df_l2_3_4)] = model.coef_
    mean_rmse_l2_3_4= mean(rmse_final)
    mean_coef_df_l2_3_4=coef_df_l2_3_4.mean().to_frame()

    std=np.std([mean_rmse_l23,mean_rmse_l4,mean_rmse_l2_3_4])
    
    return mean_rmse_l23,mean_rmse_l4,mean_rmse_l2_3_4,std,mean_coef_df_l23,mean_coef_df_l4,mean_coef_df_l2_3_4
    

In [ ]:
m4_mean_rmse_l23,m4_mean_rmse_l4,m4_mean_rmse_l2_3_4,m4_std,m4_mean_coef_df_l23,m4_mean_coef_df_l4,m4_mean_coef_df_l2_3_4=prediction_eval_function("mouse-24705_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse24705_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse24705_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse24705_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m3_mean_rmse_l23,m3_mean_rmse_l4,m3_mean_rmse_l2_3_4,m3_std,m3_mean_coef_df_l23,m3_mean_coef_df_l4,m3_mean_coef_df_l2_3_4=prediction_eval_function("mouse-24617_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse24617_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse24617_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse24617_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m5_mean_rmse_l23,m5_mean_rmse_l4,m5_mean_rmse_l2_3_4,m5_std,m5_mean_coef_df_l23,m5_mean_coef_df_l4,m5_mean_coef_df_l2_3_4=prediction_eval_function("mouse-25133_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse25133_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25133_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse25133_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m6_mean_rmse_l23,m6_mean_rmse_l4,m6_mean_rmse_l2_3_4,m6_std,m6_mean_coef_df_l23,m6_mean_coef_df_l4,m6_mean_coef_df_l2_3_4=prediction_eval_function("mouse-25341_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse25341_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25341_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse25341_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m7_mean_rmse_l23,m7_mean_rmse_l4,m7_mean_rmse_l2_3_4,m7_std,m7_mean_coef_df_l23,m7_mean_coef_df_l4,m7_mean_coef_df_l2_3_4=prediction_eval_function("mouse-25387_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse25387_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25387_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse25387_IoannisThreshold_3nz_1.5dc_full_60min.csv")


In [ ]:
#std=np.std([mean_rmse_l23,mean_rmse_l4,mean_rmse_l2_3_4])
data = {'L2/3':round((m4_mean_rmse_l23+m3_mean_rmse_l23+m5_mean_rmse_l23+m6_mean_rmse_l23+m7_mean_rmse_l23)/5,2), 'L4':round((m4_mean_rmse_l4+m3_mean_rmse_l4+m5_mean_rmse_l4+m6_mean_rmse_l4+m7_mean_rmse_l4)/5,2), 'L4 & L2/3':round((m4_mean_rmse_l2_3_4+m3_mean_rmse_l2_3_4+m5_mean_rmse_l2_3_4+m6_mean_rmse_l2_3_4+m7_mean_rmse_l2_3_4)/5,2),'Error':round((m4_std+m3_std+m5_std+m7_std+m6_std)/5,2)}
x = list(data.keys())
y = list(data.values())

fig = plt.figure(figsize = (10, 5))
# creating the bar plot
plt.bar(x, y, color ='maroon',width = 0.4)
plt.xlabel("Cases")
plt.ylabel("Accuracy/RMSE")
plt.title("Prediction Accuracy in Different Cases Across All 5 Mice") #molis mou steilei kai ta upoloipa pontikia tha valw kai to "across all 5 mice" alliws "for mouse 4"
for i in range(len(y)):
    plt.annotate(str(y[i]), xy=(x[i],y[i]), ha='center', va='bottom')
plt.show()

In [ ]:
def generate_scatterplots_coef_ndoc(coef_l23, coef_l4,coef_l23_l4,ndoc_l23,ndoc_l4_l23,ndoc_l4,mouse_name):
    
    coef_df_l23=coef_l23.rename(columns={0: "neurons"})
    coef_df_l23=coef_df_l23.rename(columns={"index": "neurons","neurons":"coef"})
    coef_df_l23=coef_df_l23.reset_index()
    coef_df_l23=coef_df_l23.rename(columns={"index": "neurons"})
    coef_df_l23['neurons']=coef_df_l23['neurons'].astype(int)
    
    ndoc_data_l23 = pd.read_csv(ndoc_l23)
    
    final2_3 = pd.merge(coef_df_l23, ndoc_data_l23, on="neurons")
    final2_3.plot.scatter(x = 'ndoc', y = 'coef')
    plt.title(mouse_name+": Case 1/a")
    plt.xlabel("NDOC L2/3")
    plt.ylabel("Impact / Coefficient L2/3")
    
    ndoc_data_l4_l23=pd.read_csv(ndoc_l4_l23)
    
    final_l2_3_4 = pd.merge(coef_df_l23, ndoc_data_l4_l23, on="neurons")
    final_l2_3_4.plot.scatter(x = 'ndoc', y = 'coef')
    plt.title(mouse_name+": Case 1/b")
    plt.xlabel("NDOC L4 -> L2/3")
    plt.ylabel("Impact / Coefficient L2/3")
    
    ndoc_data_l4_L4 = pd.read_csv(ndoc_l4)
    
    coef_df_l4=coef_l4.rename(columns={0: "neurons"})
    coef_df_l4=coef_df_l4.rename(columns={"index": "neurons","neurons":"coef"})
    coef_df_l4=coef_df_l4.reset_index()
    coef_df_l4=coef_df_l4.rename(columns={"index": "neurons"})
    coef_df_l4['neurons']=coef_df_l4['neurons'].astype(int)
    final_l4_4 = pd.merge(coef_df_l4, ndoc_data_l4_L4, on="neurons")
    final_l4_4.plot.scatter(x = 'ndoc', y = 'coef')
    plt.title(mouse_name+": Case 2")
    plt.xlabel("NDOC L4")
    plt.ylabel("Impact / Coefficient L4")
    
    coef_df_l2_3_4=coef_l23_l4.rename(columns={0: "neurons"})
    coef_df_l2_3_4=coef_df_l2_3_4.rename(columns={"index": "neurons","neurons":"coef"})
    coef_df_l2_3_4=coef_df_l2_3_4.reset_index()
    coef_df_l2_3_4=coef_df_l2_3_4.rename(columns={"index": "neurons"})
    coef_df_l2_3_4['neurons']=coef_df_l2_3_4['neurons'].astype(int)
    final_l2_3_l4_4 = pd.merge(coef_df_l2_3_4, ndoc_data_l4_L4, on="neurons")
    final_l2_3_l4_4.plot.scatter(x = 'ndoc', y = 'coef')
    plt.title(mouse_name+": Case 3")
    plt.xlabel("NDOC L4")
    plt.ylabel("Impact / Coefficient L2/3 & L4")
    
    return final2_3,final_l4_4,final_l2_3_4
   

In [ ]:
m4_l23,m4_l4,m4_l23_4=generate_scatterplots_coef_ndoc(m4_mean_coef_df_l23, m4_mean_coef_df_l4,m4_mean_coef_df_l2_3_4,'mouse4_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse4_L4_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse4_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','Mouse 4(24705)')
m3_l23,m3_l4,m3_l23_4=generate_scatterplots_coef_ndoc(m3_mean_coef_df_l23, m3_mean_coef_df_l4,m3_mean_coef_df_l2_3_4,'mouse3_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse3_L4_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse3_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','Mouse 3(24617)')
m5_l23,m5_l4,m5_l23_4=generate_scatterplots_coef_ndoc(m5_mean_coef_df_l23, m5_mean_coef_df_l4,m5_mean_coef_df_l2_3_4,'mouse5_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse5_L4_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse5_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','Mouse 5(25133)')
m6_l23,m6_l4,m6_l23_4=generate_scatterplots_coef_ndoc(m6_mean_coef_df_l23, m6_mean_coef_df_l4,m6_mean_coef_df_l2_3_4,'mouse6_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse6_L4_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse6_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','Mouse 6(25341)')
m7_l23,m7_l4,m7_l23_4=generate_scatterplots_coef_ndoc(m7_mean_coef_df_l23, m7_mean_coef_df_l4,m7_mean_coef_df_l2_3_4,'mouse7_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse7_L4_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse7_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','Mouse 7(25387)')

In [ ]:
def generate_histogram(small_sttc_groups, large_sttc_groups,mouse_mean_coef_df_l2_3_4,mouse_name):    
    small_group = pd.read_feather(small_sttc_groups)
    large_group = pd.read_feather(large_sttc_groups)
    
    small_group.drop(['STTC Group','Group Size','UnusedNeurons'], axis=1, inplace=True) 
    large_group.drop(['STTC Group','Group Size','UnusedNeurons'], axis=1, inplace=True) # dropping the cols I dont need
    large_group=large_group.rename(columns={"Obs Ref Neuron": "neurons"})
    small_group=small_group.rename(columns={"Obs Ref Neuron": "neurons"})
    
    coef_df_l2_3_4=mouse_mean_coef_df_l2_3_4.rename(columns={0: "neurons"})
    coef_df_l2_3_4=coef_df_l2_3_4.rename(columns={"index": "neurons","neurons":"coef"})
    coef_df_l2_3_4=coef_df_l2_3_4.reset_index()
    coef_df_l2_3_4=coef_df_l2_3_4.rename(columns={"index": "neurons"})
    coef_df_l2_3_4['neurons']=coef_df_l2_3_4['neurons'].astype(int)
    
    
    
        
    imp_mean_2_3_4=coef_df_l2_3_4['coef'].mean()
    imp_std_2_3_4=coef_df_l2_3_4['coef'].std()
    
    coef_df_l2_3_4['coef']=(coef_df_l2_3_4['coef']-imp_mean_2_3_4)/imp_std_2_3_4
    
    
    
    final_large = coef_df_l2_3_4[coef_df_l2_3_4['neurons'].isin(large_group['neurons'])]
    final_small=coef_df_l2_3_4[coef_df_l2_3_4['neurons'].isin(small_group['neurons'])]
        
    plt.figure(figsize=(8,6))
    plt.hist(final_small['coef'], bins=100, alpha=0.8,density=True, label="Small STTC group",color='yellow')
    plt.hist(final_large['coef'], bins=100, alpha=0.8, density=True, label="Large STTC group",color='purple')

    plt.xlabel('Z-score Impact', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse_name+' Histogram for L4->L23 STTC groups')
    plt.legend(loc='upper right')

    plt.show()

In [ ]:
generate_histogram('Mouse_3_1.5dc_0dt_L4_L23_small_sttc_groups_obs.feather','Mouse_3_1.5dc_0dt_L4_L23_large_sttc_groups_obs.feather',m3_mean_coef_df_l23,'Mouse 3(24617)')
generate_histogram('Mouse_4_1.5dc_0dt_L4_L23_small_sttc_groups_obs.feather','Mouse_4_1.5dc_0dt_L4_L23_large_sttc_groups_obs.feather',m4_mean_coef_df_l23,'Mouse 4(24705)')
generate_histogram('Mouse_5_1.5dc_0dt_L4_L23_small_sttc_groups_obs.feather','Mouse_5_1.5dc_0dt_L4_L23_large_sttc_groups_obs.feather',m5_mean_coef_df_l23,'Mouse 5(25133)')
generate_histogram('Mouse_6_1.5dc_0dt_L4_L23_small_sttc_groups_obs.feather','Mouse_6_1.5dc_0dt_L4_L23_large_sttc_groups_obs.feather',m6_mean_coef_df_l23,'Mouse 6(25341)')
generate_histogram('Mouse_7_1.5dc_0dt_L4_L23_small_sttc_groups_obs.feather','Mouse_7_1.5dc_0dt_L4_L23_large_sttc_groups_obs.feather',m7_mean_coef_df_l23,'Mouse 7(25387)')

In [ ]:
def prediction_eval_function_old_values(mouse_pupil_csv, l23_csv,l4_csv,spiketrains_csv):

    mouse_pupil= pd.read_csv(mouse_pupil_csv)
    l23 = pd.read_csv(l23_csv)
    l4=pd.read_csv(l4_csv)
    spiketrains = pd.read_csv(spiketrains_csv)
    
    spiketrains.columns=spiketrains.columns.str.replace('V', '')
    neurons2_3=list(l23.loc[:,"Neurons"])
    neurons2_3=[str(x) for x in neurons2_3]
    neurons4=list(l4.loc[:,"Neurons"])
    neurons4=[str(x) for x in neurons4]
    n23=spiketrains.drop(columns=[col for col in spiketrains.columns if col not in neurons2_3])
    n4=spiketrains.drop(columns=[col for col in spiketrains.columns if col not in neurons4])
    n23_4=spiketrains.drop(columns=[col for col in spiketrains.columns if col not in neurons4 and col not in neurons2_3 ])
    result2_3 = pd.concat([mouse_pupil, n23], axis=1, join='inner')
    result4 = pd.concat([mouse_pupil, n4], axis=1, join='inner')
    result2_3_4=pd.concat([mouse_pupil, n23_4], axis=1, join='inner')
    result2_3.drop(['Frames','Timestamps','PupilX','PupilY'], axis=1, inplace=True) # dropping the cols I dont need
    result4.drop(['Frames','Timestamps','PupilX','PupilY'], axis=1, inplace=True) # dropping the cols I dont need
    result2_3_4.drop(['Frames','Timestamps','PupilX','PupilY'], axis=1, inplace=True) # dropping the cols I dont need
    
    pupil_mean_2_3_4=result2_3_4['PupilRadius'].mean()
    pupil_std_2_3_4=result2_3_4['PupilRadius'].std()
    
    pupil_mean_2_3=result2_3['PupilRadius'].mean()
    pupil_std_2_3=result2_3['PupilRadius'].std()
    
    pupil_mean4=result4['PupilRadius'].mean()
    pupil_std4=result4['PupilRadius'].std()
    
    
    result2_3['PupilRadius']=(result2_3['PupilRadius']-pupil_mean_2_3)/pupil_std_2_3
    result4['PupilRadius']=(result4['PupilRadius']-pupil_mean4)/pupil_std4
    result2_3_4['PupilRadius']=(result2_3_4['PupilRadius']-pupil_mean_2_3_4)/pupil_std_2_3_4


    X = result2_3.iloc[:, 1:].values
    y = result2_3.iloc[:, 0].values
    kf = KFold(n_splits = 12,shuffle=True, random_state=10)
    rmse_final= list()
    columns=result2_3.iloc[:, 1:].columns.tolist()
    coef_df_l23=pd.DataFrame(columns=columns)
    for train_ix, test_ix in kf.split(X):
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        model =LinearSVR(C=0.5, dual=True, epsilon=0.0, fit_intercept=True,
            intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=10000,
            random_state=None, tol=0.0001, verbose=0)
        train_X = train_X[4:]
        train_y = train_y[:-4]
        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)
        rmse = mean_squared_error(test_y, y_pred)*(1/2.0)
        rmse_final.append(rmse)
        #print('> ', rmse)
        coef_df_l23.loc[len(coef_df_l23)] = model.coef_
    mean_rmse_l23= mean(rmse_final)
    mean_coef_df_l23=coef_df_l23.mean().to_frame()

    X = result4.iloc[:, 1:].values
    y = result4.iloc[:, 0].values
    kf = KFold(n_splits = 12,shuffle=True, random_state=10)
    rmse_final= list()
    columns=result4.iloc[:, 1:].columns.tolist()
    coef_df_l4=pd.DataFrame(columns=columns)
    for train_ix, test_ix in kf.split(X):
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        model =LinearSVR(C=0.5, dual=True, epsilon=0.0, fit_intercept=True,
            intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=10000,
            random_state=None, tol=0.0001, verbose=0)
        train_X = train_X[4:]
        train_y = train_y[:-4]
        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)
        rmse = mean_squared_error(test_y, y_pred)*(1/2.0)
        rmse_final.append(rmse)
        #print('> ', rmse)
        coef_df_l4.loc[len(coef_df_l4)] = model.coef_
    mean_rmse_l4= mean(rmse_final)
    mean_coef_df_l4=coef_df_l4.mean().to_frame()

    X = result2_3_4.iloc[:, 1:].values
    y = result2_3_4.iloc[:, 0].values
    kf = KFold(n_splits = 12,shuffle=True, random_state=10)
    rmse_final= list()
    columns=result2_3_4.iloc[:, 1:].columns.tolist()
    coef_df_l2_3_4=pd.DataFrame(columns=columns)
    for train_ix, test_ix in kf.split(X):
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        model =LinearSVR(C=0.5, dual=True, epsilon=0.0, fit_intercept=True,
            intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=10000,
            random_state=None, tol=0.0001, verbose=0)
        train_X = train_X[4:]
        train_y = train_y[:-4]
        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)
        rmse = mean_squared_error(test_y, y_pred)*(1/2.0)
        rmse_final.append(rmse)
        #print('> ', rmse)
        coef_df_l2_3_4.loc[len(coef_df_l2_3_4)] = model.coef_
    mean_rmse_l2_3_4= mean(rmse_final)
    mean_coef_df_l2_3_4=coef_df_l2_3_4.mean().to_frame()

    std=np.std([mean_rmse_l23,mean_rmse_l4,mean_rmse_l2_3_4])
    
    return mean_rmse_l23,mean_rmse_l4,mean_rmse_l2_3_4,std,mean_coef_df_l23,mean_coef_df_l4,mean_coef_df_l2_3_4

In [ ]:
m4_mean_rmse_l23,m4_mean_rmse_l4,m4_mean_rmse_l2_3_4,m4_std,m4_mean_coef_df_l23,m4_mean_coef_df_l4,m4_mean_coef_df_l2_3_4=prediction_eval_function_old_values("mouse-24705_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse24705_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse24705_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse24705_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m3_mean_rmse_l23,m3_mean_rmse_l4,m3_mean_rmse_l2_3_4,m3_std,m3_mean_coef_df_l23,m3_mean_coef_df_l4,m3_mean_coef_df_l2_3_4=prediction_eval_function_old_values("mouse-24617_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse24617_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse24617_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse24617_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m5_mean_rmse_l23,m5_mean_rmse_l4,m5_mean_rmse_l2_3_4,m5_std,m5_mean_coef_df_l23,m5_mean_coef_df_l4,m5_mean_coef_df_l2_3_4=prediction_eval_function_old_values("mouse-25133_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse25133_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25133_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse25133_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m6_mean_rmse_l23,m6_mean_rmse_l4,m6_mean_rmse_l2_3_4,m6_std,m6_mean_coef_df_l23,m6_mean_coef_df_l4,m6_mean_coef_df_l2_3_4=prediction_eval_function_old_values("mouse-25341_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse25341_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25341_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse25341_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m7_mean_rmse_l23,m7_mean_rmse_l4,m7_mean_rmse_l2_3_4,m7_std,m7_mean_coef_df_l23,m7_mean_coef_df_l4,m7_mean_coef_df_l2_3_4=prediction_eval_function_old_values("mouse-25387_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse25387_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25387_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse25387_IoannisThreshold_3nz_1.5dc_full_60min.csv")

#std=np.std([mean_rmse_l23,mean_rmse_l4,mean_rmse_l2_3_4])
data = {'L2/3':round((m4_mean_rmse_l23+m3_mean_rmse_l23+m5_mean_rmse_l23+m6_mean_rmse_l23+m7_mean_rmse_l23)/5,2), 'L4':round((m4_mean_rmse_l4+m3_mean_rmse_l4+m5_mean_rmse_l4+m6_mean_rmse_l4+m7_mean_rmse_l4)/5,2), 'L4 & L2/3':round((m4_mean_rmse_l2_3_4+m3_mean_rmse_l2_3_4+m5_mean_rmse_l2_3_4+m6_mean_rmse_l2_3_4+m7_mean_rmse_l2_3_4)/5,2)}
x = list(data.keys())
y = list(data.values())

c = ['blue', 'orange', 'green']

fig = plt.figure(figsize = (10, 5))
# creating the bar plot
plt.bar(x, y, color =c,width = 0.4)
plt.xlabel("Cases")
plt.ylabel("Accuracy/RMSE")
plt.title("Prediction Accuracy in Different Cases Across All 5 Mice, 4 Timeframes in the Past")
for i in range(len(y)):
    plt.annotate(str(y[i]), xy=(x[i],y[i]), ha='center', va='bottom')
plt.savefig('plt.png')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
def pca_function(mouse_pupil_csv, l23_csv,l4_csv,spiketrains_csv):

    mouse_pupil= pd.read_csv(mouse_pupil_csv)
    l23 = pd.read_csv(l23_csv)
    l4=pd.read_csv(l4_csv)
    spiketrains = pd.read_csv(spiketrains_csv)
    
    spiketrains.columns=spiketrains.columns.str.replace('V', '')
    neurons2_3=list(l23.loc[:,"Neurons"])
    neurons2_3=[str(x) for x in neurons2_3]
    neurons4=list(l4.loc[:,"Neurons"])
    neurons4=[str(x) for x in neurons4]
 
    n23_4=spiketrains.drop(columns=[col for col in spiketrains.columns if col not in neurons4 and col not in neurons2_3 ])

    result2_3_4=pd.concat([mouse_pupil, n23_4], axis=1, join='inner')

    result2_3_4.drop(['Frames','Timestamps','PupilX','PupilY'], axis=1, inplace=True) # dropping the cols I dont need
    
    pupil_mean_2_3_4=result2_3_4['PupilRadius'].mean()
    pupil_std_2_3_4=result2_3_4['PupilRadius'].std()
    
    result2_3_4['PupilRadius']=(result2_3_4['PupilRadius']-pupil_mean_2_3_4)/pupil_std_2_3_4


    X = result2_3_4.iloc[:, 1:].values
    y = result2_3_4.iloc[:, 0].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    pca = PCA(n_components=10)
    pca.fit(X_train)
    
    neurons_all = pd.DataFrame({'Neurons': list(result2_3_4.columns[1:])})

    return pca,neurons_all

In [ ]:
m4_pca,m4_neurons=pca_function("mouse-24705_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse24705_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse24705_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse24705_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m3_pca,m3_neurons=pca_function("mouse-24617_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse24617_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse24617_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse24617_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m5_pca,m5_neurons=pca_function("mouse-25133_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse25133_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25133_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse25133_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m6_pca,m6_neurons=pca_function("mouse-25341_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse25341_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25341_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse25341_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m7_pca,m7_neurons=pca_function("mouse-25387_behavioral_pupil_cleaned_interpolated_spontaneous_frames.csv","mouse25387_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25387_L4_neuronIDs_15um_V1_0.01Hz.csv","mouse25387_IoannisThreshold_3nz_1.5dc_full_60min.csv")


In [ ]:
def Explained_Variance_Ratio(pca,l23_csv,l4_csv,neurons,mouse_name): 
    
    explained_variance_ratio_neurons = np.sum(pca.components_**2, axis=0) / np.sum(pca.components_**2)

    # Get the top 10 neurons based on their explained variance ratio
    #top_neurons = np.argsort(explained_variance_ratio_neurons)[::-1][:len(explained_variance_ratio_neurons)]
    #top_neurons_explained_variance_ratio = explained_variance_ratio_neurons[top_neurons]
    
    l23 = pd.read_csv(l23_csv)
    l4=pd.read_csv(l4_csv)
    
    df = pd.DataFrame({'Neurons': neurons['Neurons'].astype(int), 'Power': explained_variance_ratio_neurons})
    df_l23 = pd.merge(df, l23, on='Neurons')
    df_l4 = pd.merge(df, l4, on='Neurons')
    
    return df_l23,df_l4


In [ ]:
m3_neurons_power_l23,m3_neurons_power_l4 = Explained_Variance_Ratio(m3_pca,"mouse24617_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse24617_L4_neuronIDs_15um_V1_0.01Hz.csv",m3_neurons,'Mouse 3(24617)')
m4_neurons_power_l23,m4_neurons_power_l4 = Explained_Variance_Ratio(m4_pca,"mouse24705_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse24705_L4_neuronIDs_15um_V1_0.01Hz.csv",m4_neurons,'Mouse 4(24705)')
m5_neurons_power_l23,m5_neurons_power_l4 = Explained_Variance_Ratio(m5_pca,"mouse25133_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25133_L4_neuronIDs_15um_V1_0.01Hz.csv",m5_neurons,'Mouse 5(25133)')
m6_neurons_power_l23,m6_neurons_power_l4 = Explained_Variance_Ratio(m6_pca,"mouse25341_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25341_L4_neuronIDs_15um_V1_0.01Hz.csv",m6_neurons,'Mouse 6(25341)')
m7_neurons_power_l23,m7_neurons_power_l4 = Explained_Variance_Ratio(m7_pca,"mouse25387_L23_neuronIDs_15um_V1_0.01Hz.csv","mouse25387_L4_neuronIDs_15um_V1_0.01Hz.csv",m7_neurons,'Mouse 7(25387)')

In [ ]:
def general_histograms(df_l23,df_l4,mouse):
        
    plt.figure(figsize=(8,6))
    plt.hist(df_l23['Power'], bins=100, alpha=0.8,density=True, label="Layer2-3 Neurons",color='yellow')
    plt.hist(df_l4['Power'], bins=100, alpha=0.8, density=True, label="Layer4 Neurons",color='purple')

    plt.xlabel('Prediction Impact', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' General Histogram for neurons impact in prediction')
    plt.legend(loc='upper right')
    plt.show()  


In [ ]:
general_histograms(m3_neurons_power_l23,m3_neurons_power_l4,'Mouse 3(24617)')
general_histograms(m4_neurons_power_l23,m4_neurons_power_l4,'Mouse 4(24705)')
general_histograms(m5_neurons_power_l23,m5_neurons_power_l4,'Mouse 5(25133)')
general_histograms(m6_neurons_power_l23,m6_neurons_power_l4,'Mouse 6(25341)')
general_histograms(m7_neurons_power_l23,m7_neurons_power_l4,'Mouse 7(25387)')

In [ ]:
# Create a function that receives an array of neuron IDs, a spiketrain dataset and filters it for a specific firing rate
# threshold given a sample frequency
def calculate_neuron_firing_rate(neuron_spikes_data, f_sample=6.30072):
    neuron_spikes_data = pd.read_csv(neuron_spikes_data)
    neuron_spikes_data.columns=neuron_spikes_data.columns.str.replace('V', '')

    # Get number of spikes
    spikes = np.sum(neuron_spikes_data, axis=0)

    # Get firing rate
    T_sample = 1 / f_sample
    frames = neuron_spikes_data.shape[0]
    firing_rate = spikes / (frames * T_sample)
    firing_rate= pd.DataFrame(firing_rate)
    firing_rate = firing_rate.rename_axis('neuron_id').reset_index().rename(columns={0: 'firing rate'})
    firing_rate['neuron_id']=firing_rate['neuron_id'].astype(int)
    # Return the firing rate
    return firing_rate

In [ ]:
m3_firing_rate=calculate_neuron_firing_rate("mouse24617_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m4_firing_rate=calculate_neuron_firing_rate("mouse24705_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m5_firing_rate=calculate_neuron_firing_rate("mouse25133_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m6_firing_rate=calculate_neuron_firing_rate("mouse25341_IoannisThreshold_3nz_1.5dc_full_60min.csv")
m7_firing_rate=calculate_neuron_firing_rate("mouse25387_IoannisThreshold_3nz_1.5dc_full_60min.csv")


In [ ]:
def create_df_for_plots(neurons_power_l23,neurons_power_l4,ndoc_l23,ndoc_l4,xyz,orientation_pref,fir_rate):
    
    xyz_data = pd.read_csv(xyz)
    
    or_data = pd.read_csv(orientation_pref)
    or_data.rename(columns = {'neuronID':'neuron_id'}, inplace = True)
    
    or_data.drop(['dir_pref_rad','dir_pref_degrees'], axis=1, inplace=True) # dropping the cols I dont need

    
    ndoc_data_l23 = pd.read_csv(ndoc_l23)
    ndoc_data_l4 = pd.read_csv(ndoc_l4)
    
    ndoc_data_l23.rename(columns = {'neurons':'neuron_id'}, inplace = True)
    ndoc_data_l4.rename(columns = {'neurons':'neuron_id'}, inplace = True)

    neurons_power_l23.rename(columns = {'Neurons':'neuron_id'}, inplace = True)
    neurons_power_l4.rename(columns = {'Neurons':'neuron_id'}, inplace = True)
    
    neurons_power_xyz_l23 = pd.merge(neurons_power_l23, xyz_data, on='neuron_id')
    neurons_power_xyz_l4 = pd.merge(neurons_power_l4, xyz_data, on='neuron_id')
    
    neurons_power_xyz_or_l23 = pd.merge(neurons_power_xyz_l23, or_data, on='neuron_id')
    neurons_power_xyz_or_l4 = pd.merge(neurons_power_xyz_l4, or_data, on='neuron_id')    
    
    neurons_power_xyz_or_ndoc_l23 = pd.merge(neurons_power_xyz_or_l23, ndoc_data_l23, on='neuron_id')
    neurons_power_xyz_or_ndoc_l4 = pd.merge(neurons_power_xyz_or_l4, ndoc_data_l4, on='neuron_id')
    
    neurons_power_xyz_or_ndoc_fir_l23 = pd.merge(neurons_power_xyz_or_ndoc_l23, fir_rate, on='neuron_id')
    neurons_power_xyz_or_ndoc_fir_l4 = pd.merge(neurons_power_xyz_or_ndoc_l4, fir_rate, on='neuron_id')
    
    return neurons_power_xyz_or_ndoc_fir_l23.sort_values(by='Power', ascending=False), neurons_power_xyz_or_ndoc_fir_l4.sort_values(by='Power', ascending=False)

In [ ]:
m3_df_for_plots_l23,m3_df_for_plots_l4 = create_df_for_plots(m3_neurons_power_l23,m3_neurons_power_l4,'mouse3_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse3_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse24617_coords.csv','mouse24617_s3_idx10_strongest_amplitude_peak_all_cells.csv',m3_firing_rate)
m4_df_for_plots_l23,m4_df_for_plots_l4 = create_df_for_plots(m4_neurons_power_l23,m4_neurons_power_l4,'mouse4_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse4_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse24705_coords.csv','mouse24705_s3_idx24_strongest_amplitude_peak_all_cells.csv',m4_firing_rate)
m5_df_for_plots_l23,m5_df_for_plots_l4 = create_df_for_plots(m5_neurons_power_l23,m5_neurons_power_l4,'mouse5_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse5_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse25133_coords.csv','mouse25133_s6_idx15_strongest_amplitude_peak_all_cells.csv',m5_firing_rate)
m6_df_for_plots_l23,m6_df_for_plots_l4 = create_df_for_plots(m6_neurons_power_l23,m6_neurons_power_l4,'mouse6_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse6_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse25341_coords.csv','mouse25341_s2_idx20_strongest_amplitude_peak_all_cells.csv',m6_firing_rate)
m7_df_for_plots_l23,m7_df_for_plots_l4 = create_df_for_plots(m7_neurons_power_l23,m7_neurons_power_l4,'mouse7_L23_L23_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse3_L4_L4_correlated_DoC_1.5dc_0dt_15um_0.01Hz_V1.csv','mouse25387_coords.csv','mouse25387_s3_idx17_strongest_amplitude_peak_all_cells.csv',m7_firing_rate)

In [ ]:
def split_neurons(l23,l4):
    threshold = l23['Power'].quantile(q=0.95)

    # create a dataframe containing the top 5% based on 'Score' column
    l23_top5 = l23[l23['Power'] >= threshold]

    # create a dataframe containing the rest of the data
    l23_rest = l23[l23['Power'] < threshold]
    
    threshold = l4['Power'].quantile(q=0.95)

    # create a dataframe containing the top 5% based on 'Score' column
    l4_top5 = l4[l4['Power'] >= threshold]

    # create a dataframe containing the rest of the data
    l4_rest = l4[l4['Power'] < threshold]
    
    
    return l23_top5,l23_rest,l4_top5,l4_rest

In [ ]:
m3_l23_top5,m3_l23_rest,m3_l4_top5,m3_l4_rest=split_neurons(m3_df_for_plots_l23,m3_df_for_plots_l4)
m4_l23_top5,m4_l23_rest,m4_l4_top5,m4_l4_rest=split_neurons(m4_df_for_plots_l23,m4_df_for_plots_l4)
m5_l23_top5,m5_l23_rest,m5_l4_top5,m5_l4_rest=split_neurons(m5_df_for_plots_l23,m5_df_for_plots_l4)
m6_l23_top5,m6_l23_rest,m6_l4_top5,m6_l4_rest=split_neurons(m6_df_for_plots_l23,m6_df_for_plots_l4)
m7_l23_top5,m7_l23_rest,m7_l4_top5,m7_l4_rest=split_neurons(m7_df_for_plots_l23,m7_df_for_plots_l4)

In [ ]:
num_rows = int(len(m4_df_for_plots_l4) * 0.05)
index_top5 = len(m4_df_for_plots_l4) - num_rows

# create a dataframe containing the top 5% based on 'Score' column
m4_l4_top5 = m4_df_for_plots_l4.iloc[index_top5:, :]

# create a dataframe containing the rest of the data
m4_l4_rest = m4_df_for_plots_l4.iloc[:index_top5, :]

In [ ]:
def histograms_xyz(l23_top5,l23_rest,l4_top5,l4_rest,mouse):
    plt.figure(figsize=(8,6))
    plt.hist(l23_top5['x'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l23_rest['x'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('X coordinate', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' X coordinate Layer 2-3')
    plt.legend(loc='upper right')
    plt.show()  
    
    plt.figure(figsize=(8,6))
    plt.hist(l4_top5['x'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l4_rest['x'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('X coordinate', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' X coordinate Layer 4')
    plt.legend(loc='upper right')
    plt.show()  
    
    plt.figure(figsize=(8,6))
    plt.hist(l23_top5['y'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l23_rest['y'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('Y coordinate', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' Y coordinate Layer 2-3')
    plt.legend(loc='upper right')
    plt.show()  
    
    plt.figure(figsize=(8,6))
    plt.hist(l4_top5['y'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l4_rest['y'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('Y coordinate', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' Y coordinate Layer 4')
    plt.legend(loc='upper right')
    plt.show() 
    
    plt.figure(figsize=(8,6))
    plt.hist(l23_top5['z'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l23_rest['z'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('Z coordinate', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' Z coordinate Layer 2-3')
    plt.legend(loc='upper right')
    plt.show()  
    
    plt.figure(figsize=(8,6))
    plt.hist(l4_top5['z'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l4_rest['z'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('Z coordinate', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' Z coordinate Layer 4')
    plt.legend(loc='upper right')
    plt.show() 

In [ ]:
histograms_xyz(m3_l23_top5,m3_l23_rest,m3_l4_top5,m3_l4_rest,'Mouse 3(24617)')
histograms_xyz(m4_l23_top5,m4_l23_rest,m4_l4_top5,m4_l4_rest,'Mouse 4(24705)')
histograms_xyz(m5_l23_top5,m5_l23_rest,m5_l4_top5,m5_l4_rest,'Mouse 5(25133)')
histograms_xyz(m6_l23_top5,m6_l23_rest,m6_l4_top5,m6_l4_rest,'Mouse 6(25341)')
histograms_xyz(m7_l23_top5,m7_l23_rest,m7_l4_top5,m7_l4_rest,'Mouse 7(25387)')

In [ ]:
def histograms_or_pref(l23_top5,l23_rest,l4_top5,l4_rest,mouse):
    plt.figure(figsize=(8,6))
    plt.hist(l23_top5['pvalue'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l23_rest['pvalue'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('Pvalue', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' All neurons orientation preference Layer 2-3')
    plt.legend(loc='upper right')
    plt.show()  
    
    plt.figure(figsize=(8,6))
    plt.hist(l4_top5['pvalue'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l4_rest['pvalue'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('Pvalue', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' All neurons orientation preference Layer 4')
    plt.legend(loc='upper right')
    plt.show()
    
    l23_top5 = l23_top5[(l23_top5['pvalue'] <= 0.001) & (l23_top5['von_pred_adv'] >= 0.025)]
    l23_rest = l23_rest[(l23_rest['pvalue'] <= 0.001) & (l23_rest['von_pred_adv'] >= 0.025)]
    l4_top5 = l4_top5[(l23_top5['pvalue'] <= 0.001) & (l4_top5['von_pred_adv'] >= 0.025)]
    l4_rest = l4_rest[(l4_rest['pvalue'] <= 0.001) & (l4_rest['von_pred_adv'] >= 0.025)]
    
    plt.figure(figsize=(8,6))
    plt.hist(l23_top5['pvalue'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l23_rest['pvalue'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('Pvalue', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' Orientation selective neurons orientation preference Layer 2-3')
    plt.legend(loc='upper right')
    plt.show()  
    
    plt.figure(figsize=(8,6))
    plt.hist(l4_top5['pvalue'], bins=100, alpha=0.8,density=True, label="Top 5% Neurons",color='yellow')
    plt.hist(l4_rest['pvalue'], bins=100, alpha=0.8, density=True, label="Rest Neurons",color='purple')

    plt.xlabel('Pvalue', size=14)
    plt.ylabel("Count", size=14)
    plt.title(mouse+' Orientation selective neurons orientation preference Layer 4')
    plt.legend(loc='upper right')
    plt.show()    

In [ ]:
histograms_or_pref(m3_l23_top5,m3_l23_rest,m3_l4_top5,m3_l4_rest,'Mouse 3(24617)')
histograms_or_pref(m4_l23_top5,m4_l23_rest,m4_l4_top5,m4_l4_rest,'Mouse 4(24705)')
histograms_or_pref(m5_l23_top5,m5_l23_rest,m5_l4_top5,m5_l4_rest,'Mouse 5(25133)')
histograms_or_pref(m6_l23_top5,m6_l23_rest,m6_l4_top5,m6_l4_rest,'Mouse 6(25341)')
histograms_or_pref(m7_l23_top5,m7_l23_rest,m7_l4_top5,m7_l4_rest,'Mouse 7(25387)')

In [ ]:
def scatterplot_ndoc(l23_top5,l23_rest,l4_top5,l4_rest,mouse):
    
    plt.scatter(l23_top5['Power'], l23_top5['ndoc'])
    plt.xlabel('Predictive Power ')
    plt.ylabel('Ndoc')
    plt.title(mouse+' Layer2-3 Top 5% Neurons Scatter Plot Power/Ndoc ')
    plt.show()
    
    plt.scatter(l23_rest['Power'], l23_rest['ndoc'])
    plt.xlabel('Predictive Power ')
    plt.ylabel('Ndoc')
    plt.title(mouse+' Layer2-3 Rest Neurons Scatter Plot Power/Ndoc ')
    plt.show()
    
    plt.scatter(l4_top5['Power'], l4_top5['ndoc'])
    plt.xlabel('Predictive Power ')
    plt.ylabel('Ndoc')
    plt.title(mouse+' Layer4 Top 5% Neurons Scatter Plot Power/Ndoc ')
    plt.show()
    
    plt.scatter(l4_rest['Power'], l4_rest['ndoc'])
    plt.xlabel('Predictive Power ')
    plt.ylabel('Ndoc')
    plt.title(mouse+' Layer4 Rest Neurons Scatter Plot Power/Ndoc ')
    plt.show()

In [ ]:
scatterplot_ndoc(m3_l23_top5,m3_l23_rest,m3_l4_top5,m3_l4_rest,'Mouse 3(24617)')
scatterplot_ndoc(m4_l23_top5,m4_l23_rest,m4_l4_top5,m4_l4_rest,'Mouse 4(24705)')
scatterplot_ndoc(m5_l23_top5,m5_l23_rest,m5_l4_top5,m5_l4_rest,'Mouse 5(25133)')
scatterplot_ndoc(m6_l23_top5,m6_l23_rest,m6_l4_top5,m6_l4_rest,'Mouse 6(25341)')
scatterplot_ndoc(m7_l23_top5,m7_l23_rest,m7_l4_top5,m7_l4_rest,'Mouse 7(25387)')

In [ ]:
def scatterplot_firing_rate(l23_top5,l23_rest,l4_top5,l4_rest,mouse):
    
    plt.scatter(l23_top5['Power'], l23_top5['firing rate'])
    plt.xlabel('Predictive Power ')
    plt.ylabel('firing rate')
    plt.title(mouse+' Layer2-3 Top 5% Neurons Scatter Plot Power/firing rate ')
    plt.show()
    
    plt.scatter(l23_rest['Power'], l23_rest['firing rate'])
    plt.xlabel('Predictive Power ')
    plt.ylabel('firing rate')
    plt.title(mouse+' Layer2-3 Rest Neurons Scatter Plot Power/firing rate ')
    plt.show()
    
    plt.scatter(l4_top5['Power'], l4_top5['firing rate'])
    plt.xlabel('Predictive Power ')
    plt.ylabel('firing rate')
    plt.title(mouse+' Layer4 Top 5% Neurons Scatter Plot Power/firing rate ')
    plt.show()
    
    plt.scatter(l4_rest['Power'], l4_rest['firing rate'])
    plt.xlabel('Predictive Power ')
    plt.ylabel('firing rate')
    plt.title(mouse+' Layer4 Rest Neurons Scatter Plot Power/firing rate ')
    plt.show()

In [ ]:
scatterplot_firing_rate(m3_l23_top5,m3_l23_rest,m3_l4_top5,m3_l4_rest,'Mouse 3(24617)')
scatterplot_firing_rate(m4_l23_top5,m4_l23_rest,m4_l4_top5,m4_l4_rest,'Mouse 4(24705)')
scatterplot_firing_rate(m5_l23_top5,m5_l23_rest,m5_l4_top5,m5_l4_rest,'Mouse 5(25133)')
scatterplot_firing_rate(m6_l23_top5,m6_l23_rest,m6_l4_top5,m6_l4_rest,'Mouse 6(25341)')
scatterplot_firing_rate(m7_l23_top5,m7_l23_rest,m7_l4_top5,m7_l4_rest,'Mouse 7(25387)')